**D3APL: Aplicações em Ciência de Dados** <br/>
IFSP Campinas

Prof. Dr. Samuel Martins (Samuka) <br/><br/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

# Implementing a Binary Linear Classifier

## Logistic Regression with GD and L2 Regularization (Ridge)

PS: use the lecture slides to support your development.

### L2 Regularization
$$
R(\mathbf{w}) = \alpha\frac{1}{2}\sum_{j=1}^{n}w_j^2
$$

### Gradient Descent

Nothing changes for the bias.
$$
b = b - \eta \sum_{i=1}^{m}\left[(\hat{p}^{(i)} - y^{(i)})\right]
$$

$$
w_j = w_j - \eta \left( \sum_{i=1}^{m}\left[(\hat{p}^{(i)} - y^{(i)}) x^{(i)}_j \right] + \alpha w_j \right)
$$

## 1. Set up

#### Imports

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#### Loading fake data

In [ ]:
X_train = np.load('./datasets/fake_train.npy')
y_train = np.load('./datasets/fake_train_labels.npy')

X_test = np.load('./datasets/fake_test.npy')
y_test = np.load('./datasets/fake_test_labels.npy')

In [ ]:
print(f'X_train.shape = {X_train.shape}')
print(f'y_train.shape = {y_train.shape}')

print(f'X_test.shape = {X_test.shape}')
print(f'y_test.shape = {y_test.shape}')

In [ ]:
import seaborn as sns

sns.scatterplot(x=X_train[:, 0], y=X_train[:, 1], hue=y_train)
plt.xlabel('x1')
plt.xlabel('x2')
plt.title('Scatter plot: Training Samples')

In [ ]:
import seaborn as sns

sns.scatterplot(x=X_test[:, 0], y=X_test[:, 1], hue=y_test, marker='s')
plt.xlabel('x1')
plt.xlabel('x2')
plt.title('Scatter plot: Testing Samples')

## 2. Implementation
### Plan of Attack
- Add the regularization parameter $\alpha$ to the constructor
- Change \_\_str\_\_
- Change the gradient function

In [ ]:
from typing import Tuple

import numpy as np
from numpy import ndarray

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y


class LogisticRegression(ClassifierMixin, BaseEstimator):
    """Our Logistic Regression implemented from scratch."""
    
    def __init__(self, learning_rate : float = 0.001,
                 n_epochs : int = 1000, random_state : int = 42):
        """
        Parameters
        ----------
        learning_rate : float, default=0.001
            Learning rate.
        n_epochs : int, default=1000
            Number of epochs for training (convergence stop).
        random_state : int, default=42
            Seed used for generating random numbers.
        """
        assert (learning_rate is not None) and (learning_rate > 0.0), \
        f'Learning rate must be > 0. Passed: {learning_rate}'

        assert (n_epochs is not None) and n_epochs > 0, \
        f'Number of epochs must be > 0. Passed: {n_epochs}'
        
        assert (random_state is not None) and random_state >= 0, \
        f'Random state should be >= 0. Passed: {random_state}'

        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.random_state = random_state

        # model / parameters - private
        self.__w = None
        # bias
        self.__b = None
    
    
    # a special method used to represent a class object as a string, called with print() or str()
    def __str__(self):
        msg = f'Learning Rate: {self.learning_rate}\n' \
              f'Number of Epochs: {self.n_epochs}\n' \
              f'Random state (seed): {self.random_state}\n\n' \
              f'Trained?: {self.is_fitted()}\n'
        return msg
    
    
    def is_fitted(self) -> bool:
        """Check if the estimator is fitted by verifying the presence of a (learned) weight matrix"""
        return self.__w is not None

    
    # getter: access the function as an attribute
    @property
    def coef_(self) -> ndarray:
        """Return the weight matrix (learned parameters) if the estimator was fitted/trained.
           Otherwise, raise an exception.
        """
        assert self.is_fitted(), 'The instance is not fitted yet.'
        return self.__w
    
    # getter: access the function as an attribute
    @property
    def intercept_(self) -> float:
        """Return the bias (learned intercepet) if the estimator was fitted/trained.
           Otherwise, raise an exception.
        """
        assert self.is_fitted(), 'The instance is not fitted yet.'
        return self.__b

    
    def __sigmoid(self, z: float) -> float:
        return 1 / (1 + np.e ** (-z))


    def __log_loss(self, y: ndarray, p_hat: ndarray, eps: float = 1e-15):
        """Return the log loss for a given estimation and ground-truth (true labels).
        
        Since log loss is undefined for `p=0` and `p=1`, we clipped the probabilities to max(eps, min(1 - eps, p)).
        
        Parameters
        ----------
        y : ndarray, shape (n_samples,)
            True labels of input samples.
        p_hat : ndarray
            Estimated probabilities of input samples.
        eps : float, default=1e-15
            Epsilon term used to avoid undefined log loss at 0 and 1.
        
        Returns
        -------
        log_loss : float
            Computed log loss.
        """
        p_hat_eps = np.maximum(eps, np.minimum(1 - eps, p_hat))
        
        losses = -(y * np.log(p_hat_eps) + (1 - y) * np.log(1 - p_hat_eps))
        log_loss = losses.mean()

        return log_loss
    
    
    def __gradient(self, X: ndarray, y: ndarray, p_hat: ndarray) -> Tuple[ndarray, float]:
        '''Compute the gradient vector for the log loss with regards to the weights and bias.
        
        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features)
            Training data.
        y: ndarray of shape (n_samples,).
            Target (true) labels.
        p_hat : ndarray, shape (n_samples,)
            Estimated probabilities.
        
        Returns
        -------
        Tuple[ndarray, float]: 
            Tuple with:
            - a numpy array of shape (n_features,) containing the partial derivatives w.r.t. the weights; and
            - a float representing the partial derivative w.r.t. the bias.
        '''
        # X.shape ==> (n_samples, n_features)
        # y.shape ==> (n_samples,)
        n_samples = X.shape[0]
        
        error = p_hat - y  # shape (n_samples,)
        
        grad_w = np.dot(error, X) / n_samples  # shape (n_features,)
        
        # as x_0 will be array 1.0 (bias trick), the gradient for the bias is simplified
        grad_b = np.sum(error) / n_samples  # scalar
        
        return grad_w, grad_b



    def fit(self, X: ndarray, y: ndarray, verbose: int = 0):
        '''Train a Logistic Regression classifier.

        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features)
            Training data.
        y: ndarray of shape (n_samples,).
            Target (true) labels.
        verbose: int, default=0
            Verbose flag. Print training information every `verbose` iterations.
            
        Returns
        -------
        self : object
            Returns self.
        '''
        ### CHECK INPUT ARRAY DIMENSIONS
        assert X.ndim == 2, f'X must be 2D. Passed: {X.ndim}'
        assert y.ndim == 1, f'y must be 1D. Passed: {y.ndim}'
        assert X.shape[0] == y.shape[0], \
            f'X.shape[0] should be equal to y.shape[0], instead: {X.shape[0]} != {y.shape[0]}'
        # X, y = check_X_y(X, y)

        ### SETTING SEED
        np.random.seed(self.random_state)
        
        n_samples, n_features = X.shape

        # Altough the bias trick is an elegant solution to merge all
        # parameters, it requires to extend the feature matrix X
        # by adding one column of 1.0. This demands time, so we will
        # deal with the weights and bias separately.

        ### PARAMETER INITIALIZATION
        
        # return values from the “standard normal” distribution.
        w = np.random.randn(n_features)
        b = 0.0

        losses = []

        # learning iterations
        for epoch in np.arange(self.n_epochs):
            ### ESTIMATION (FORWARD PASS)
            # X.shape == (n_samples, n_features)
            # w.shape = (n_features,)
            # p_hat = (n_samples,)
            z = np.dot(X, w) + b
            p_hat = self.__sigmoid(z)
            
            loss = self.__log_loss(y, p_hat)
            losses.append(loss)
            
            ### GRADIENT DESCENT (BACKWARD PASS)
            # grad_w.shape ==> (n_samples, 1)
            # grad_b: scalar
            grad_w, grad_b = self.__gradient(X, y, p_hat)
            w = w - self.learning_rate * grad_w
            b = b - self.learning_rate * grad_b
            
            # check to see if an update should be displayed
            if verbose and (epoch == 0 or (epoch + 1) % verbose == 0):
                print(f'[INFO] epoch={epoch + 1}/{self.n_epochs}, loss={loss:.7f}')
        
        if verbose > 0:
            losses = np.array(losses)
            print(f'\nFinal loss: {loss}')
            print(f'\nMean loss: {losses.mean()} +- {losses.std()}')
            
        ### ASSIGN THE TRAINED PARAMETERS TO THE PRIVATE ATTRIBUTES
        self.__w = w
        self.__b = b
        
    
    
    def predict_proba(self, X: ndarray) -> ndarray:
        '''Estimate the probability for the positive class of input samples.

        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features)
            Input samples.
            
        Returns
        -------
        ndarray of shape (n_samples,)
            The estimated probabilities for the positive class of input samples.
        '''
        assert self.is_fitted(), 'The instance is not fitted yet.'
        assert X.ndim == 2, f'X must be 2D. Passed: {X.ndim}'
        
        z = np.dot(X, self.__w) + self.__b

        return self.__sigmoid(z)

    
    def predict(self, X: ndarray) -> ndarray:
        '''Predict the labels for input samples.
        
        Thresholding at probability >= 0.5.

        Parameters
        ----------
        X: ndarray of shape (n_samples, n_features)
            Input samples.
            
        Returns
        -------
        ndarray of shape (n_samples,)
            Predicted labels of input samples.
        '''
        assert self.is_fitted(), 'The instance is not fitted yet.'
        assert X.ndim == 2, f'X must be 2D. Passed: {X.ndim}'

        p_hat = self.predict_proba(X)
        y_hat = p_hat >= 0.5  # ndarray of bools
        y_hat = y_hat.astype(np.int)
        
        return y_hat
    
    
    

#### **Testing constructor and `__str__`**
- evaluate the default hyperparameters
- try different valid values for them
- try invalid values for them

In [ ]:
clf = LogisticRegression()

print('Printing object by print()')
print(clf)

In [ ]:
print('Displaying object')
clf

#### **Testing `fit()`**
PS: use `pdb.set_trace()` inside the main loop of `fit()` for debugging.

In [ ]:
# no regularization


In [ ]:
# L2 regularization - alpha=0.01


In [ ]:
# L2 regularization - alpha=0.0001


In [ ]:
# L2 regularization - alpha=100


#### **Visualizing the Decision Boundary**
L2 regularization with alpha=100, n_epochs=10000.

$w_1x_1 + w_2x_2 + b = 0$

$x_2 = -(b + w_1x_1)/w_2$

In [ ]:
b = clf.intercept_
w1, w2 = clf.coef_

In [ ]:
x1_decision_line = np.array([X_train[:,0].min(), X_train[:,0].max()])
x2_decision_line = -(b + (w1 * x1_decision_line)) / w2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.scatterplot(x=X_train[:, 0], y=X_train[:, 1], hue=y_train)
sns.lineplot(x=x1_decision_line, y=x2_decision_line, color='lightseagreen')
plt.xlabel('x1')
plt.xlabel('x2')
plt.title('Decision Boundary on Training Samples')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.scatterplot(x=X_test[:, 0], y=X_test[:, 1], hue=y_test, marker='s')
sns.lineplot(x=x1_decision_line, y=x2_decision_line, color='lightseagreen')
plt.xlabel('x1')
plt.xlabel('x2')
plt.title('Decision Boundary on Testing Samples')

### Prediction

In [ ]:
y_test_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

# Exercise
Evaluate different instances of our classifier with the **Breast Cancer dataset**, [avaible on sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer).

Suggestion for the experiments:
- Fix a given seed (random_state) for reproducibility
- Use 80% of the data for training, and 20% for testing - stratified sample
- Compared methods:
    - Our implementation with default parameters
    - Our implementation after fine-tuning the `learning_rate`, `n_epochs`, and `alpha`
    - [LogisticRegression from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) (which is not implemented with Grad. Descent)
    - [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier.fit) with default parameters
- Compute (at least) the F1-scores

**PS:** Only optimize our method against the default (non-optimized) baselines is not a fair comparison. One should also optimize at least the main hyperparameters from the baselines for a more fair comparison. But, this is a simple exercise! Don't worry about that.